# imports

In [1]:
from DbConnector import DbConnector
import pandas as pd
import numpy as np
import geopy.distance

# Preparation

In [2]:
connection = DbConnector()
db_connection = connection.db_connection
cursor = connection.cursor

Connected to: 8.0.30-0ubuntu0.22.04.1
You are connected to the database: ('sec_assignment',)
-----------------------------------------------



# 1. How many users, activities and trackpoints are there in the dataset (after it is inserted into the database)

In [3]:
tables = ["User", "Activity", "TrackPoint"]

for table_name in tables:
    querry = f"SELECT COUNT(id) FROM {table_name}"
    cursor.execute(querry)
    result = cursor.fetchall()

    print(f"amount of {table_name.lower()}s in table {table_name}: {result[0][0]}")

amount of users in table User: 182
amount of activitys in table Activity: 22482
amount of trackpoints in table TrackPoint: 4034966


# 2. Find the average number of activities per user. 

In [4]:
querry = "SELECT COUNT(id) / COUNT(DISTINCT(user_id)) FROM Activity a"
cursor.execute(querry)
result = cursor.fetchall()

print(f"average number of activities per user: {result[0][0]}")

average number of activities per user: 123.5275


# 3. Find the top 20 users with the highest number of activities. 

In [7]:
querry = "SELECT user_id, COUNT(id) from Activity a GROUP BY a.user_id ORDER BY COUNT(a.id) DESC LIMIT 20"
cursor.execute(querry)
result = cursor.fetchall()

for i, user in enumerate(result):
    print(f"{i+1}. id = {user[0]} (amount of activities = {user[1]})")

1. id = 163 (amount of activities = 3182)
2. id = 085 (amount of activities = 1298)
3. id = 153 (amount of activities = 1123)
4. id = 068 (amount of activities = 969)
5. id = 167 (amount of activities = 944)
6. id = 128 (amount of activities = 937)
7. id = 062 (amount of activities = 782)
8. id = 025 (amount of activities = 757)
9. id = 041 (amount of activities = 557)
10. id = 075 (amount of activities = 509)
11. id = 126 (amount of activities = 468)
12. id = 010 (amount of activities = 434)
13. id = 052 (amount of activities = 425)
14. id = 084 (amount of activities = 422)
15. id = 004 (amount of activities = 395)
16. id = 017 (amount of activities = 391)
17. id = 140 (amount of activities = 380)
18. id = 003 (amount of activities = 322)
19. id = 179 (amount of activities = 319)
20. id = 030 (amount of activities = 296)


# 4. Find all users who have taken a taxi. 

In [6]:
querry = "SELECT DISTINCT(user_id) from Activity WHERE transportation_mode = 'taxi'"
cursor.execute(querry)
result = cursor.fetchall()

for i, user in enumerate(result):
    print(f"{i}: id {user[0]} took a taxi")

0: id 010 took a taxi
1: id 020 took a taxi
2: id 021 took a taxi
3: id 052 took a taxi
4: id 056 took a taxi
5: id 058 took a taxi
6: id 062 took a taxi
7: id 065 took a taxi
8: id 068 took a taxi
9: id 075 took a taxi
10: id 078 took a taxi
11: id 080 took a taxi
12: id 082 took a taxi
13: id 084 took a taxi
14: id 085 took a taxi
15: id 091 took a taxi
16: id 098 took a taxi
17: id 100 took a taxi
18: id 102 took a taxi
19: id 104 took a taxi
20: id 105 took a taxi
21: id 111 took a taxi
22: id 114 took a taxi
23: id 118 took a taxi
24: id 126 took a taxi
25: id 128 took a taxi
26: id 139 took a taxi
27: id 147 took a taxi
28: id 153 took a taxi
29: id 154 took a taxi
30: id 161 took a taxi
31: id 163 took a taxi
32: id 167 took a taxi
33: id 175 took a taxi
34: id 179 took a taxi


# 5. Find all types of transportation modes and count how many activities that are tagged with these transportation mode labels. Do not count the rows where the mode is null. 

In [7]:
querry = """

SELECT COUNT(a.transportation_mode), a.transportation_mode 
from Activity a
JOIN User u ON a.user_id = u.id
WHERE u.has_labels = 1
GROUP BY transportation_mode

"""
cursor.execute(querry)
result = cursor.fetchall()

for i, transportation_mode in enumerate(result):
    print(f"{i}: {transportation_mode[1]} -> {transportation_mode[0]} times taken")

0: bus -> 2853 times taken
1: train -> 299 times taken
2: taxi -> 1179 times taken
3: walk -> 6460 times taken
4: subway -> 813 times taken
5: airplane -> 17 times taken
6: car -> 993 times taken
7: bike -> 2089 times taken
8: boat -> 7 times taken
9: run -> 6 times taken
10: motorcycle -> 2 times taken


# 6.1. Find the year with the most activities.

In [8]:
querry = "SELECT YEAR(start_date_time), COUNT(id) from Activity a GROUP BY YEAR(start_date_time) ORDER BY COUNT(id) DESC LIMIT 1"
cursor.execute(querry)
result = cursor.fetchall()

for i, res in enumerate(result):
    print(f"{i}: {res[0]} was the year with the most activities ({res[1]}).")

0: 2008 was the year with the most activities (10943).


# 6.2. Is this also the year with most recorded hours?

In [9]:
querry = """

SELECT YEAR(start_date_time), SUM(TIMESTAMPDIFF(HOUR, start_date_time, end_date_time)) AS diff 
FROM Activity 
GROUP BY YEAR(start_date_time)
 ORDER BY diff DESC LIMIT 1

"""
cursor.execute(querry)
result = cursor.fetchall()

for i, res in enumerate(result):
    print(f"{i}: {res[0]} was the year with the most recorded hours ({res[1]}).")

0: 2009 was the year with the most recorded hours (18791).


# 7. Find the total distance (in km) walked in 2008, by user with id=112.

In [10]:
querry = """

SELECT tp.lat, tp.lon, a.id
FROM TrackPoint tp
JOIN Activity a ON tp.activity_id = a.id
WHERE a.user_id = 112 AND YEAR(a.start_date_time) = 2008 AND a.transportation_mode = 'walk'

"""
cursor.execute(querry)
result = cursor.fetchall()

total_distance = 0
for i in range(len(result) - 1):
    if result[i][2] == result[i+1][2]:
        total_distance += geopy.distance.geodesic((result[i][0], result[i][1]),  (result[i+1][0], result[i+1][1])).km

print(f"The total disctance walked by user with id 112 in 2008 is {total_distance} km")

The total disctance walked by user with id 112 in 2008 is 109.15471726077064 km


# 8. Find the top 20 users who have gained the most altitude meters.  
- Output should be a table with (id, total meters gained per user).   
- Remember that some altitude-values are invalid  
- Tip:  ∑ (𝑡𝑝 𝑛.𝑎𝑙𝑡𝑖𝑡𝑢𝑑𝑒−𝑡𝑝 𝑛−1.𝑎𝑙𝑡𝑖𝑡𝑢𝑑𝑒), 𝑡𝑝 𝑛.  𝑎𝑙𝑡𝑖𝑡𝑢𝑑𝑒 >
𝑡𝑝 𝑛−1.𝑎𝑙𝑡𝑖𝑡𝑢𝑑𝑒

In [11]:
querry = """

SELECT a.user_id, 
tp.altitude
FROM TrackPoint tp
JOIN Activity a ON tp.activity_id = a.id
WHERE tp.altitude <> -777
ORDER BY a.user_id ASC
"""
cursor.execute(querry)
result = cursor.fetchall()

In [12]:
df = pd.DataFrame(result, columns=["id", "elevation"])
df["meter_gained"] = df.groupby(by=["id"]).diff()

df = df.groupby(by=["id"]).sum().sort_values(by=["meter_gained"], ascending=False).head(20).reset_index()
df.drop(["elevation"], axis=1, inplace=True)
df.rename(columns={"id": "id", "meter_gained": "total meters gained per user"}, inplace=True)
print("20 users who have gained the most altitude meters:")
df

20 users who have gained the most altitude meters:


,id,total meters gained per user
0,111,3777.0
1,144,2103.0
2,101,1630.0
3,055,699.0
4,172,611.0
5,064,487.0
6,067,336.0
7,041,322.0
8,163,300.0
9,108,298.0


# 9. Find all users who have invalid activities, and the number of invalid activities per user  
- An invalid activity is defined as an activity with consecutive trackpoints 
where the timestamps deviate with at least 5 minutes. 

In [13]:
querry = """

WITH cte(act_id, date_time, diff_in_time)
AS(
    SELECT activity_id,
    date_time,
    TIMESTAMPDIFF(MINUTE, LAG(date_time, 1) OVER (ORDER BY date_time), date_time)
    FROM TrackPoint
)

SELECT DISTINCT(a.id),
a.user_id,
cte.date_time,
cte.diff_in_time
FROM Activity a
JOIN cte ON cte.act_id = a.id
JOIN TrackPoint tp ON a.id = tp.activity_id
WHERE cte.diff_in_time > 5 OR cte.diff_in_time < -5

"""

cursor.execute(querry)
result = cursor.fetchall()

In [14]:
df = pd.DataFrame(result, columns=["activity_id", "id", "time", "diff_in_time"])
df.drop(["time", "diff_in_time"], axis=1, inplace=True)
df.drop_duplicates(keep="first", inplace=True)
dict = df.groupby(by=["id"]).count().reset_index().set_index('id').T.to_dict('list')   # 010 -> 88, 020 -> 85

print("-------------------------------------------------------")
print(f"There are {len(dict)} users with invalid activities")
print("-------------------------------------------------------")

for id in dict:
    print(f"user with id {id} has {dict[id][0]} invalid activities.")


-------------------------------------------------------
There are 164 users with invalid activities
-------------------------------------------------------
user with id 000 has 62 invalid activities.
user with id 001 has 23 invalid activities.
user with id 002 has 58 invalid activities.
user with id 003 has 108 invalid activities.
user with id 004 has 196 invalid activities.
user with id 005 has 22 invalid activities.
user with id 006 has 8 invalid activities.
user with id 007 has 11 invalid activities.
user with id 008 has 5 invalid activities.
user with id 009 has 10 invalid activities.
user with id 010 has 74 invalid activities.
user with id 011 has 56 invalid activities.
user with id 012 has 26 invalid activities.
user with id 013 has 35 invalid activities.
user with id 014 has 68 invalid activities.
user with id 015 has 16 invalid activities.
user with id 016 has 7 invalid activities.
user with id 017 has 96 invalid activities.
user with id 018 has 14 invalid activities.
user with

# 10. Find the users who have tracked an activity in the Forbidden City of Beijing.  
- In this question you can consider the Forbidden City to have coordinates that correspond to: lat 39.916, lon 116.397.

In [15]:
querry = """

SELECT DISTINCT(a.user_id)
FROM Activity a
JOIN TrackPoint tp ON a.id = tp.activity_id
WHERE tp.lat BETWEEN 39.915 AND 39.917 AND tp.lon BETWEEN 116.396 AND 116.398

"""

cursor.execute(querry)
result = cursor.fetchall()


In [16]:
pd.DataFrame(result, columns=["user_id"])

,user_id
0,004
1,018


# 11.  Find all users who have registered transportation_mode and their most used transportation_mode.  
- The answer should be on format (user_id, most_used_transportation_mode) sorted on user_id. 
- Some users may have the same number of activities tagged with e.g. walk and car. In this case it is up to you to decide which transportation mode to include in your answer (choose one). 
- Do not count the rows where the mode is null. 

In [17]:
querry = """

SELECT a.user_id,
COUNT(a.transportation_mode),
a.transportation_mode
from Activity a
JOIN User u ON a.user_id = u.id
WHERE u.has_labels = 1
GROUP BY a.user_id, a.transportation_mode
ORDER BY COUNT(a.transportation_mode) DESC

"""

cursor.execute(querry)
result = cursor.fetchall()

In [18]:
df = pd.DataFrame(result, columns=["user_id", "amount_used", "transportation_mode"])
df.drop_duplicates(keep="first", subset=["user_id"], inplace=True)
df.sort_values(by=["user_id"], ascending=True, inplace=True)

dict = df.set_index('user_id').T.to_dict('list')
for id in dict:
    print(f"user with id {id} most used transportation_mode is {dict[id][1]} ({dict[id][0]} times)")

user with id 010 most used transportation_mode is walk (153 times)
user with id 020 most used transportation_mode is bike (102 times)
user with id 021 most used transportation_mode is car (10 times)
user with id 052 most used transportation_mode is bus (222 times)
user with id 053 most used transportation_mode is walk (14 times)
user with id 056 most used transportation_mode is bike (22 times)
user with id 058 most used transportation_mode is walk (11 times)
user with id 059 most used transportation_mode is walk (1 times)
user with id 060 most used transportation_mode is walk (2 times)
user with id 062 most used transportation_mode is bus (338 times)
user with id 064 most used transportation_mode is walk (24 times)
user with id 065 most used transportation_mode is bike (106 times)
user with id 067 most used transportation_mode is walk (63 times)
user with id 068 most used transportation_mode is bike (573 times)
user with id 069 most used transportation_mode is walk (8 times)
user with 

In [19]:
df.drop(["amount_used"], axis=1, inplace=True)
df

,user_id,transportation_mode
25,010,walk
35,020,bike
125,021,car
17,052,bus
108,053,walk
...,...,...
7,167,walk
130,170,walk
196,174,car
180,175,walk
